In [59]:
import pandas as pd
import numpy as np
import re

from urllib.parse import urlparse
import contractions
import string
import num2words

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [60]:
train_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/train_subtask1.csv')
train_df

,index,text,label
0,train_01_0_892,The State alleged they hacked Sabata Petros Ch...,1
1,train_01_1_2714,Chale was allegedly chased by a group of about...,0
2,train_01_10_2619,The farmworkers ' strike resumed on Tuesday wh...,1
3,train_01_100_2680,Demonstrators have filed for a permit to hold ...,1
4,train_01_101_3090,"Footage of the attack , which included a pregn...",1
...,...,...,...
3070,train_09_B_95_1244,She said the group had to be dispersed after t...,1
3071,train_09_B_96_3017,"The remaining person in the group , arrested d...",0
3072,train_09_B_97_3381,Tuesday was marked by further protests and pol...,0
3073,train_09_B_98_880,Tirupur : Protests against lack of amenities N...,1


In [61]:
print(train_df.isna().sum())
print(train_df.shape)
print(train_df.dtypes)
# print(train_df.label.value_counts())

index    0
text     0
label    0
dtype: int64
(3075, 3)
index    object
text     object
label     int64
dtype: object


In [62]:
abbreviations = {
    'n\'t':'not',
    'won\'t':'will not',
    'can\'t':'can not',
    'i\'m':'i am',
    'you\'re':'you are',
    'he\'s':'he is',
    'she\'s':'she is',
    'it\'s':'it is',
    'we\'re':'we are',
    'they\'re':'they are',
    'i\'ve':'i have',
    'you\'ve':'you have',
    'we\'ve':'we have',
    'NaMo':'Narendra Modi',
    'PMO':'Prime Minister Office',
    'PM':'Prime Minister',
    'BJP':'Bharatiya Janata Party',
    'INC':'Indian National Congress',
    'AAP':'Aam Aadmi Party',
    'J&K':'Jammu and Kashmir',
    'NHAI':'National Highways Authority of India',
    'CPI':'Communist Party of India',
    'CPI(M)':'Communist Party of India (Marxist)',
    'CPI-M':'Communist Party of India (Marxist)',
    'HQs':'Headquarters',
    'NGO':'Non Governmental Organization',
    'NGOs':'Non Governmental Organizations',
}


In [63]:
def removing_links(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    x = [word for word in tokens if not urlparse(word).scheme]
    text = ' '.join(x)
    return text

def contractions_handling(text):
    text = contractions.fix(text)
    return text

def replacing_abbr(text, dictry):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    x = [dictry[word] if word in dictry.keys() else word for word in tokens]
    text = ' '.join(x)
    return text

def removing_punctuations(text):
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))
    text = text.lower()
    return text

def removing_stopwords(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    x = [word for word in tokens if word not in stopwords.words('english')]
    text = ' '.join(x)
    return text

def convert_num_words(text):
    re_results = re.findall('(\d+(st|nd|rd|th))', text)
    for entire_result, suffix in re_results:
        num = int(entire_result[:-len(suffix)])
        entire_result = " " + entire_result
        num_word = num2words.num2words(num, ordinal=True)
        text = text.replace(entire_result, " " + num_word)
    
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [num2words.num2words(word) if word.isdigit() else word for word in tokens]
    text = ' '.join(tokens)
    
    return text

def removing_char_less_3(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [word for word in tokens if len(word) >= 3]
    text = ' '.join(tokens)
    return text

In [67]:
train_df['text'] = train_df['text'].apply(lambda x: removing_links(x))
train_df['text'] = train_df['text'].apply(lambda x: contractions_handling(x))
# train_df['text'] = train_df['text'].apply(lambda x: replacing_abbr(x, abbreviations))
train_df['text'] = train_df['text'].apply(lambda x: removing_punctuations(x))
train_df['text'] = train_df['text'].apply(lambda x: removing_stopwords(x))
train_df['text'] = train_df['text'].apply(lambda x: convert_num_words(x))
train_df['text'] = train_df['text'].apply(lambda x: removing_char_less_3(x))

train_df.head()

,index,text,label
0,train_01_0_892,state alleged hacked sabata petros chale thirt...,1
1,train_01_1_2714,chale allegedly chased group thirty people hac...,0
2,train_01_10_2619,farmworkers strike resumed tuesday demands met,1
3,train_01_100_2680,demonstrators filed permit hold rally saturday...,1
4,train_01_101_3090,footage attack included pregnant woman hit pro...,1


In [66]:
train_df.to_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/train_subtask1_preprocessed_3.csv', index=False)

In [ ]:
# 1 - no stopwords and removed few abbreviations
# 2 - with stopwords and removed few abbreviations
# 3 - no stopwords and didnt remove abbreviations
# 4 - with stopwords and didnt remove abbreviations